In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from PIL import Image 

In [ ]:
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

In [ ]:
learning_rate = 1e-6
meta_step_size = 0.25 #0.25

inner_batch_size = 8
eval_batch_size = 8

meta_iters = 1000 # 2000
eval_iters = 5 # repetitions
inner_iters = 3 # repetitions

eval_interval = 1 #meta_iter % eval_interval == 0이면 eval. 1이면 매번 한다는 뜻.
train_shots = 431
f_shots = 5 # eval
classes = 5
split = 5

In [ ]:
class Dataset:
    # This class will facilitate the creation of a few-shot dataset
    # from the Omniglot dataset that can be sampled from quickly while also
    # allowing to create new labels at the same time.
    def __init__(self, mode):
        # Download the tfrecord files containing the omniglot data and convert to a
        # dataset.
        self.mode = mode
        self.data = {}
                
        for label in range(11):
            path = os.path.join(os.getcwd(), 'image10', self.mode, str(label))
            files = os.listdir(path)
            for f in files:
                image = Image.open(path + "/" + f)  # uint8 image
                image = np.asarray(image).astype('float32')
                image /= 255.0
                if label not in self.data:
                    self.data[label] = []
                self.data[label].append(image)
                self.labels = list(self.data.keys())
    
    def get_mini_dataset(
        self, batch_size, repetitions, shots, num_classes, split=False
    ):
        temp_labels = np.zeros(shape=(num_classes * shots))
        temp_images = np.zeros(shape=(num_classes * shots, 331, 331, 3))
        if split:
            test_labels = np.zeros(shape=(num_classes * split))
            test_images = np.zeros(shape=(num_classes * split, 331, 331, 3))

        # Get a random subset of labels from the entire label set.
        label_subset = random.sample(self.labels, k=num_classes)
        for class_idx, class_obj in enumerate(label_subset):
            # Use enumerated index value as a temporary label for mini-batch in
            # few shot learning.
            temp_labels[class_idx * shots : (class_idx + 1) * shots] = class_idx
            # If creating a split dataset for testing, select an extra sample from each
            # label to create the test dataset.
            if split:
                test_labels[class_idx * split : (class_idx + 1) * split] = class_idx
                images_to_split = random.choices(
                    self.data[label_subset[class_idx]], k=shots + split
                )
                test_images[
                    class_idx * split : (class_idx + 1) * split
                ] = images_to_split[-split]
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = images_to_split[:-split]
            else:
                # For each index in the randomly selected label_subset, sample the
                # necessary number of images.
                temp_images[
                    class_idx * shots : (class_idx + 1) * shots
                ] = random.choices(self.data[label_subset[class_idx]], k=shots)

        dataset = tf.data.Dataset.from_tensor_slices(
            (temp_images.astype(np.float32), temp_labels.astype(np.int32))
        )
        dataset = dataset.shuffle(shots).batch(batch_size).repeat(repetitions)
        if split:
            return dataset, test_images.astype(np.float32), test_labels.astype(np.int32)
        return dataset


import urllib3

urllib3.disable_warnings()  # Disable SSL warnings that may happen during download.
train_dataset = Dataset(mode='train')
test_dataset = Dataset(mode='val')

In [ ]:
img_rows, img_cols, img_channel = 331, 331, 3

base_model = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [ ]:
x = tf.keras.layers.Flatten()(base_model)
outputs = tf.keras.layers.Dense(classes, activation="softmax")(x)
model = tf.keras.Model(inputs=base_model.input, outputs=outputs)
model.compile()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

In [ ]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(tf.keras.layers.Dense(units=classes, activation=tf.nn.softmax))

model = tf.keras.Model(inputs=base_model.input, outputs=add_model(base_model.output))
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
#model.summary()

In [ ]:
training = []
testing = []
for meta_iter in range(meta_iters):
    frac_done = meta_iter / meta_iters
    cur_meta_step_size = (1 - frac_done) * meta_step_size
    # Temporarily save the weights from the model.
    old_vars = model.get_weights()
    # Get a sample from the full dataset.
    mini_dataset = train_dataset.get_mini_dataset(
        inner_batch_size, inner_iters, train_shots, classes
    )
    for images, labels in mini_dataset:
        with tf.GradientTape() as tape:
            preds = model(images)
            loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
    new_vars = model.get_weights()
    # Perform SGD for the meta step.
    for var in range(len(new_vars)):
        new_vars[var] = old_vars[var] + (
            (new_vars[var] - old_vars[var]) * cur_meta_step_size
        )
    # After the meta-learning step, reload the newly-trained weights into the model.
    model.set_weights(new_vars)
    # Evaluation loop
    if meta_iter % eval_interval == 0:
        accuracies = []
        for dataset in (train_dataset, test_dataset):
            # Sample a mini dataset from the full dataset.
            train_set, test_images, test_labels = dataset.get_mini_dataset(
                eval_batch_size, eval_iters, f_shots, classes, split=split
            )
            old_vars = model.get_weights()
            # Train on the samples and get the resulting accuracies.
            for images, labels in train_set:
                with tf.GradientTape() as tape:
                    preds = model(images)
                    loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
                grads = tape.gradient(loss, model.trainable_weights)
                optimizer.apply_gradients(zip(grads, model.trainable_weights))
            test_preds = model.predict(test_images)
            print(test_preds)
            test_preds = tf.argmax(test_preds, axis=1).numpy()
            num_correct = (test_preds == test_labels).sum()
            # Reset the weights after getting the evaluation accuracies.
            model.set_weights(old_vars)
            accuracies.append(num_correct / (classes*split))
        training.append(accuracies[0])
        testing.append(accuracies[1])
        #if meta_iter % 100 == 0:
        print(
            "batch %d: train=%f test=%f" % (meta_iter, accuracies[0], accuracies[1])
        )

In [ ]:
# First, some preprocessing to smooth the training and testing arrays for display.
window_length = 100
train_s = np.r_[
    training[window_length - 1 : 0 : -1], training, training[-1:-window_length:-1]
]
test_s = np.r_[
    testing[window_length - 1 : 0 : -1], testing, testing[-1:-window_length:-1]
]
w = np.hamming(window_length)
train_y = np.convolve(w / w.sum(), train_s, mode="valid")
test_y = np.convolve(w / w.sum(), test_s, mode="valid")

# Display the training accuracies.
x = np.arange(0, len(test_y), 1)
plt.plot(x, test_y, x, train_y)
plt.legend(["test", "train"])
plt.grid()
plt.show()

In [ ]:
train_set, test_images, test_labels = dataset.get_mini_dataset(
    eval_batch_size, eval_iters, f_shots, classes, split=split
)
for images, labels in train_set:
    with tf.GradientTape() as tape:
        preds = model(images)
        loss = keras.losses.sparse_categorical_crossentropy(labels, preds)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
test_preds = model.predict(test_images)
test_preds = tf.argmax(test_preds).numpy()
num_correct = (test_preds == test_labels).sum()
print(num_correct / (classes*split))

In [ ]:
#model.save("1000_fs_model_f_m.h5")

In [ ]:
model2= tf.keras.Model(inputs=model.input, outputs=model.layers[-2].output)

In [ ]:
add_model = tf.keras.Sequential()
add_model.add(tf.keras.layers.Flatten(input_shape=model2.output_shape[1:]))
add_model.add(tf.keras.layers.Dropout(rate = 0.8))
add_model.add(tf.keras.layers.Dense(units=11, activation=tf.nn.softmax))

In [ ]:
f_model = tf.keras.Model(inputs=model2.input, outputs=add_model(model2.output))
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
f_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
batch_size = 8
epochs = 1000

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255.)

train_set = train_datagen.flow_from_directory('image10/train',
                                                 target_size = (331, 331),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')

val_set = val_datagen.flow_from_directory('image10/val',
                                                 target_size = (331, 331),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')


callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = f_model.fit_generator(
    train_set,
    steps_per_epoch= 4742 // batch_size,
    epochs=epochs,
    validation_data=val_set,
    validation_steps = 527 // batch_size,
    callbacks=[callback]
)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('model auc')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

In [ ]:
test_set = test_datagen.flow_from_directory('image10/test',
                                                 target_size = (331, 331),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical')
score = f_model.evaluate(test_set, steps = 28501 // batch_size)

In [ ]:
test_set = test_datagen.flow_from_directory('image10/test',
                                                 target_size = (331, 331),
                                                 batch_size = batch_size,
                                                 class_mode = 'categorical',
                                                shuffle=False)
preds = f_model.predict(test_set, steps = 28501 // batch_size)
a = test_set.classes

In [ ]:
y_test = np.zeros((a.size, a.max()+1))
y_test[np.arange(a.size),a] = 1

In [ ]:
y_test = y_test[:-(28501%batch_size)]

In [ ]:
def LIFT(preds, y_test, cls): # >=2.5
    condition = y_test.astype(bool) #preds에 정답인 예측 score만 남긴다.
    c = np.extract(condition, preds)
    b = np.argsort(-c)[:len(c)//5] #예측 score 상위 20%
    
    lift_20 = preds[b] 
    
    lift_20_flat = np.argmax(lift_20, axis=1) #값을 확률에서 0, 1값으로 바꾼다. 
    y_test_flat = np.argmax(y_test, axis=1)
    lift_20_1 = lift_20_flat[lift_20_flat == cls] #1인 경우만 남긴다. 
    y_1 = y_test_flat[y_test_flat == cls]
    
    lift_score = (len(lift_20_1)/len(lift_20_flat))/(len(y_1)/len(y_test_flat))
    print('LIFT Accuracy: ',  lift_score)
    return lift_score

In [ ]:
lift_score = [0, 0, 0]
lift_score[0] = LIFT(preds, y_test, 3)
lift_score[1] = LIFT(preds, y_test, 4)
lift_score[2] = LIFT(preds, y_test, 7)
avg_lift = sum(lift_score) / 3

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

fpr = dict()
tpr = dict()
roc_auc = dict()
for i in [3, 4, 7]:
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], preds[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
avg_auroc = sum([roc_auc[3], roc_auc[4], roc_auc[7]]) / 3

In [ ]:
final_score = (avg_lift)*0.7 + (avg_auroc)*0.3
print(avg_lift, avg_auroc, final_score)

In [ ]:
#model.save('fianl_model.h5')